In [15]:
%%configure -f
{"conf": {"spark.jars": "/jar/simpleHTM.jar,/jar/jhealpix.jar"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1576183992464_0004,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1576183992464_0004,spark,idle,Link,Link,✔


In [16]:
import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf

//var htmindex= new HTMindex()
//val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

//val htmidUDF=udf(htmid)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf


In [17]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object HTMUtils


In [25]:
object HEALPixUtils extends Serializable {
    var hp = new Healpix() with Serializable
    val healpixid: (Double, Double) => Long = hp.ang2pix(_,_)

    val healpixidUDF=udf(healpixid)
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object HEALPixUtils


In [20]:
val dataset = Seq((123.0,45.0), (124.0,46.0)).toDF("ra", "dec")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataset: org.apache.spark.sql.DataFrame = [ra: double, dec: double]


In [21]:
dataset.withColumn("htmid", htmidUDF(dataset("ra"),dataset("dec"))).show

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<console>:30: error: not found: value htmidUDF
       dataset.withColumn("htmid", htmidUDF(dataset("ra"),dataset("dec"))).show
                                   ^



In [22]:
val datafile = "/user/hive/warehouse/source_test_parquet"
val old_df = spark.read.parquet(datafile)

old_df.createOrReplaceTempView("source")

val coordsDF = spark.sql("select coord_ra, coord_decl from source limit 10")
coordsDF.show()
coordsDF.printSchema()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datafile: String = /user/hive/warehouse/source_test_parquet
old_df: org.apache.spark.sql.DataFrame = [id: bigint, chunkid: int ... 86 more fields]
coordsDF: org.apache.spark.sql.DataFrame = [coord_ra: double, coord_decl: double]
+------------------+------------------+
|          coord_ra|        coord_decl|
+------------------+------------------+
|347.75390124573187|34.006256639533504|
| 347.7212959235707| 34.00378336432932|
|347.72357395382915| 34.00364889406109|
|347.79822646854007| 33.92442947660814|
| 347.7756980478584|34.058467401092614|
|347.80243738281285|  33.9174136042188|
| 347.5995288819927| 34.03270754662762|
| 347.6018936340388| 34.06582719619188|
| 347.6079022209228| 34.07212750950642|
| 347.6079630382091|  34.0718343187988|
+------------------+------------------+

root
 |-- coord_ra: double (nullable = true)
 |-- coord_decl: double (nullable = true)



In [23]:
val new_df = coordsDF.withColumn("htmid",HTMUtils.htmidUDF(old_df("coord_ra"),old_df("coord_decl")))
new_df.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

new_df: org.apache.spark.sql.DataFrame = [coord_ra: double, coord_decl: double ... 1 more field]
+------------------+-----------------+--------------+
|          coord_ra|       coord_decl|         htmid|
+------------------+-----------------+--------------+
|357.03448646379684|33.41077013684323|13319061131005|
+------------------+-----------------+--------------+
only showing top 1 row



In [26]:
val new_df = coordsDF.withColumn("htmid",HEALPixUtils.healpixidUDF(old_df("coord_ra"),old_df("coord_decl")))
new_df.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

new_df: org.apache.spark.sql.DataFrame = [coord_ra: double, coord_decl: double ... 1 more field]
+------------------+-----------------+-------------+
|          coord_ra|       coord_decl|        htmid|
+------------------+-----------------+-------------+
|354.76947419685087|33.73542757602207|5479435600527|
+------------------+-----------------+-------------+
only showing top 1 row

